In [9]:
from pdbfixer import PDBFixer
import simtk.openmm as mm
from simtk.openmm.app import PDBFile, Modeller, ForceField

import nglview as nv
from math import sqrt

## Fixing the pdb file of Estrogen receptor

In [2]:
receptor_file = "../1_system_inspection/receptor_1.pdb"
view = nv.show_structure_file(receptor_file)
view

NGLWidget()

In [3]:
# First load the file 
fixer = PDBFixer(filename=receptor_file)
type(fixer)

pdbfixer.pdbfixer.PDBFixer

In [4]:
# help(PDBFixer)

In [4]:
fixer.findMissingResidues()
missing_residues = fixer.missingResidues
print(f"{len(missing_residues)} missing residues")

fixer.findNonstandardResidues()
nonstandard_residues = fixer.nonstandardResidues
print(f"{len(missing_residues)} non standard residues")

fixer.findMissingAtoms()
missing_atoms = fixer.missingAtoms
missing_terminals = fixer.missingTerminals
print(f"{len(missing_atoms)} missing atoms")
print(f"{len(missing_terminals)} missing terminals")
fixer.addMissingAtoms()

0 missing residues
0 non standard residues
3 missing atoms
0 missing terminals


In [13]:
forcefield = ForceField('amber14-all.xml', 'amber14/tip3pfb.xml')
# Protonate and solvate. Convert first to Modeller
modeller = Modeller(fixer.topology, fixer.positions)
pH = 7.0
modeller.addHydrogens(forcefield=forcefield, pH=pH)

# Vectors to define the octahedron
vectors = mm.Vec3(1,0,0), mm.Vec3(1/3,2*sqrt(2)/3,0), mm.Vec3(-1/3,sqrt(2)/3,sqrt(6)/3)
# boxVectors = [(maxSize+geompadding)*v for v in vectors]
modeller.addSolvent(forcefield=forcefield, model="tip3p", boxVectors=vectors, neutralize=False)

In [14]:
PDBFile.writeFile(modeller.topology, modeller.positions, open('fixed_receptor.pdb', 'w'))